In [38]:
%cat a.txt

python 中国
python 爬虫
python 自动化运维
python WEB开发


In [39]:
string = '我是增加的字符'
tmp =[]
flag = False
with open('./a.txt','r+') as f:
    line = f.readline()
    while line:
        if '爬虫' in line:
            c = (f.tell() - len(line.encode()))
            flag = True
            source = line.strip('\n')
        elif flag:
            tmp.append(line)
        line = f.readline()
    f.seek(0)    
    f.seek(c)
    write = source+string +'\n'+ ''.join(tmp)
    f.writelines(write)

In [40]:
%cat a.txt

python 中国
python 爬虫我是增加的字符
python 自动化运维
python WEB开发


In [15]:
class Student(object):
    def __call__(self,name,score):
        self.name=name
        self.score=score
        self.print_score()
        self.get_grade()
    
    def print_score(self):
        print('%s : %s' % (self.name,self.score))
        
    def get_grade(self):
        if self.score>=90:
            print( 'A')
        elif self.score >=60:
            print( 'B')
        else:
            print( 'c') 

In [17]:
Student()('a',1)

a : 1
c


In [18]:
c = Student()

In [19]:
c('a',1)

a : 1
c


In [47]:
import contextlib

In [56]:
@contextlib.contextmanager
def context():
    print('start')
    print('a')
    try:
        raise ValueError('ABC')
    finally:
        print('123')
        print('haah')
        
        print("exit")

In [57]:
with context():
    print('haha')

start
a
123
haah
exit


ValueError: ABC

In [63]:
from functools import wraps
class ContextManager:
    def __init__(self,fn):
        self.fun = fn
    def __call__(self,*args,**kwargs):
        return self.fun(*args,**kwargs)
    def __enter__(self):
        try:
            print(next(self.__call__()))
        except:
            pass
    def __exit__(self,exc_type,exc,traceback):
        print(exc_type)
        print(exc)
        print(traceback)

In [64]:
@ContextManager
def context2():
    print('start')
    print('a')
    try:
        raise ValueError("abc")
        yield "abc"
    finally:
        print("exit")

In [65]:
with context2:
    print('haha')

start
a
exit
haha
None
None
None


In [8]:
class C:
    def __enter__(self):
        print('haha')
    def __exit__(self,exc_type,exc,traceback):
        print('exit')
        

In [9]:
with C():
    print('ha')

haha
ha
exit


In [16]:
%cat a.txt

python 中国
python 爬虫hahaha
 自动化运维
python WEB开发
